# 코로나 확진자 데이터 & 기상청 데이터 이용하여 확진자 수와 날씨 상관관계 분석

## 1. 확진자 데이터 파싱 (XML)
+ 이용한 데이터 : https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15043376
+ 인증키 : %2FargzrCJK5%2BwZ0DhHr2rbJYbgS%2Bgrj9W2jtM45tBMXuSmZQkjpSezFTK4hUtq65ZuvcfgdpfjvKw1iqAfaDRaw%3D%3D

## 2. 날씨 데이터 (csv) 파싱하기

In [95]:
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote


# 1. URL 파라미터 분리하기.
# Service URL
xmlUrl = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'

My_API_Key = unquote('%2FargzrCJK5%2BwZ0DhHr2rbJYbgS%2Bgrj9W2jtM45tBMXuSmZQkjpSezFTK4hUtq65ZuvcfgdpfjvKw1iqAfaDRaw%3D%3D')    # 아래 내가 받은 인증키가 안 되서 수업용 인증키 사용.
# My_API_Key = unquote('Agq7hySmyMi1FFU9kYibP%2BEnxYepQ%2FB6Dn%2Bw9lsYKVSCDjTwIdvpjmuhJrtyQrhipg3F3a4jbSq%2FLxHi%2FdUIoQ%3D%3D')    # 사용자 인증키
queryParams = '?' + urlencode(    # get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것이다. 메타코드 아님.
    { 
      quote_plus('ServiceKey') : My_API_Key, 
      quote_plus('pageNo') : '1',
      quote_plus('numOfRows') : '10',
      quote_plus('startCreateDt') : '20200310',
      quote_plus('endCreateDt') : '20210315' }
)

response = requests.get(xmlUrl + queryParams).text.encode('utf-8')
xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')


In [96]:
response

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><accDefRate>1.3690604347</accDefRate><accExamCnt>7084940</accExamCnt><accExamCompCnt>7013350</accExamCompCnt><careCnt>6588</careCnt><clearCnt>87754</clearCnt><createDt>2021-03-15 09:51:09.37</createDt><deathCnt>1675</deathCnt><decideCnt>96017</decideCnt><examCnt>71590</examCnt><resutlNegCnt>6917333</resutlNegCnt><seq>449</seq><stateDt>20210315</stateDt><stateTime>00:00</stateTime><updateDt>null</updateDt></item><item><accDefRate>1.3668864032</accDefRate><accExamCnt>7066401</accExamCnt><accExamCompCnt>6996558</accExamCompCnt><careCnt>6558</careCnt><clearCnt>87408</clearCnt><createDt>2021-03-14 09:42:57.64</createDt><deathCnt>1669</deathCnt><decideCnt>95635</decideCnt><examCnt>69843</examCnt><resutlNegCnt>6900923</resutlNegCnt><seq>448</seq><stateDt>20210314</stateDt><stateTime>00:00</stateTime><updateDt>null</updateDt></i

In [97]:
rows = xmlobj.findAll('item')

In [98]:
rows[0]

<item><accDefRate>1.3690604347</accDefRate><accExamCnt>7084940</accExamCnt><accExamCompCnt>7013350</accExamCompCnt><careCnt>6588</careCnt><clearCnt>87754</clearCnt><createDt>2021-03-15 09:51:09.37</createDt><deathCnt>1675</deathCnt><decideCnt>96017</decideCnt><examCnt>71590</examCnt><resutlNegCnt>6917333</resutlNegCnt><seq>449</seq><stateDt>20210315</stateDt><stateTime>00:00</stateTime><updateDt>null</updateDt></item>

In [99]:
# 한 개 행의 모든 컬럼값을 리스트에 담아보자.
columns = rows[0].find_all()
columns

[<accDefRate>1.3690604347</accDefRate>,
 <accExamCnt>7084940</accExamCnt>,
 <accExamCompCnt>7013350</accExamCompCnt>,
 <careCnt>6588</careCnt>,
 <clearCnt>87754</clearCnt>,
 <createDt>2021-03-15 09:51:09.37</createDt>,
 <deathCnt>1675</deathCnt>,
 <decideCnt>96017</decideCnt>,
 <examCnt>71590</examCnt>,
 <resutlNegCnt>6917333</resutlNegCnt>,
 <seq>449</seq>,
 <stateDt>20210315</stateDt>,
 <stateTime>00:00</stateTime>,
 <updateDt>null</updateDt>]

In [100]:
# 모든 행과 열의 값을 모아 매트릭스로 만들어보자.
rowList = []
nameList = []
columnList = []

rowsLen = len(rows)
for i in range(0, rowsLen):
    columns = rows[i].find_all()
    
    columnsLen = len(columns)
    for j in range(0, columnsLen):
        # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장한다. (어차피 rows[0], rows[1], ... 모두 컬럼헤더는 동일한 값을 가지기 때문에 매번 반복할 필요가 없다.)
        if i == 0:
            nameList.append(columns[j].name)
        # 컬럼값은 모든 행의 값을 저장해야한다.    
        eachColumn = columns[j].text
        columnList.append(eachColumn)
    rowList.append(columnList)
    columnList = []    # 다음 row의 값을 넣기 위해 비워준다. (매우 중요!!)
    
result = pd.DataFrame(rowList, columns=nameList)

In [101]:
result

,accDefRate,accExamCnt,accExamCompCnt,careCnt,clearCnt,createDt,deathCnt,decideCnt,examCnt,resutlNegCnt,seq,stateDt,stateTime,updateDt
0,1.3690604347,7084940,7013350,6588,87754,2021-03-15 09:51:09.37,1675,96017,71590,6917333,449,20210315,00:00,null
1,1.3668864032,7066401,6996558,6558,87408,2021-03-14 09:42:57.64,1669,95635,69843,6900923,448,20210314,00:00,null
2,1.3637663657,7046782,6978908,6884,86625,2021-03-13 09:36:41.886,1667,95176,67874,6883732,447,20210313,00:00,null
3,1.3635801994,7012664,6943926,7281,85743,2021-03-12 09:49:58.077,1662,94686,68738,6849240,446,20210312,00:00,null
4,1.3634619468,6976985,6908737,7871,84675,2021-03-11 09:39:02.159,1652,94198,68248,6814539,445,20210311,00:00,null
5,1.3646582727,6938884,6868606,7773,84312,2021-03-10 09:45:29.181,1648,93733,70278,6774873,444,20210310,00:00,null
6,1.3649942487,6902984,6832483,7718,83900,2021-03-09 09:52:46.909,1645,93263,70501,6739220,443,20210309,00:00,null
7,1.3660778742,6861809,6794415,7701,83474,2021-03-08 09:31:46.646,1642,92817,67394,6701598,442,20210308,00:00,null
8,1.3645371735,6843126,6776730,7617,83220,2021-03-07 09:52:34.246,1634,92471,66396,6684259,441,20210307,00:00,null
9,1.3624109264,6821943,6756772,7510,82913,2021-03-06 09:40:17.621,1632,92055,65171,6664717,440,20210306,00:00,null


In [108]:
# 이상한 데이터 값 바꾸기
for i in range(len(result['decideCnt'])-1):
  prev = result.iloc[i]['decideCnt']
  after = result.iloc[i+1]['decideCnt']
  if after < prev:
    print(prev, after)
result['decideCnt']= result['decideCnt'].replace(['39417','38746'],['38746','39417'])

96017 95635
95635 95176
95176 94686
94686 94198
94198 93733
93733 93263
93263 92817
92817 92471
92471 92055
92055 91637
91637 91239
91239 90815
90815 90371
90371 90027
90027 89672
89672 89317
89317 88902
88902 88515
88515 88120
88120 87680
87680 87324
87324 86992
86992 86576
86576 86128
86128 85567
85567 84946
84946 84325
84325 83868
83868 83525
83525 83199
83199 82837
82837 82434
82434 81930
81930 81486
81486 81183
81183 80895
80895 80524
80524 80131
80131 79761
79761 79310
79310 78843
78843 78508
78508 78203
78203 77848
77848 77395
77395 76926
76926 76429
76429 75870
75870 75521
75521 75084
75084 74692
74692 74261
74261 73916
73916 73517
73517 73114
73114 72729
72729 72610
72610 71820
71820 71240
71240 70728
70728 70204
70204 69650
69650 69114
69114 68663
68663 67999
67999 67358
67358 66684
66684 65816
65816 64978
64978 64264
64264 63244
63244 62587
62587 61767
61767 60739
60739 59772
59772 58722
58722 57678
57678 56871
56871 55901
55901 54769
54769 53529
53529 52544
52544 51454
5145

## 3. 서울 날씨 데이터 뽑기

In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [37]:
import os


plt.rc("font", family="Malgun Gothic")

# 마이너스 폰트 깨지는 거
plt.rc("axes", unicode_minus = False)

In [38]:
# retina 설정해서 글씨 조금 선명하게
%config inlineBackend.figure_format = "retina"

In [39]:
temp = pd.read_csv("OBS_ASOS_DD_20210323145944 (1).csv", encoding="euc-kr")
temp.shape

(35242, 8)

In [40]:
temp.head()

,지점,지점명,일시,평균기온(°C),최저기온(°C),최저기온 시각(hhmi),최고기온(°C),최고기온 시각(hhmi)
0,90,속초,2020-03-10,7.6,5.0,2205.0,10.1,250.0
1,90,속초,2020-03-11,6.3,2.2,518.0,10.5,1449.0
2,90,속초,2020-03-12,10.8,4.1,217.0,17.0,1526.0
3,90,속초,2020-03-13,6.8,0.7,2255.0,11.9,1011.0
4,90,속초,2020-03-14,5.6,-0.6,532.0,11.7,1427.0


In [41]:
ex_type = temp['지점명'] == '서울'
df_seoul = temp[ex_type]
df_seoul

,지점,지점명,일시,평균기온(°C),최저기온(°C),최저기온 시각(hhmi),최고기온(°C),최고기온 시각(hhmi)
4081,108,서울,2020-03-10,6.4,3.2,2343.0,10.8,1.0
4082,108,서울,2020-03-11,3.8,0.0,658.0,8.3,1454.0
4083,108,서울,2020-03-12,6.8,0.2,638.0,13.3,1435.0
4084,108,서울,2020-03-13,4.9,-0.1,2400.0,8.7,1427.0
4085,108,서울,2020-03-14,3.6,-2.7,642.0,8.6,1440.0
...,...,...,...,...,...,...,...,...
4447,108,서울,2021-03-11,10.2,4.7,720.0,17.1,1436.0
4448,108,서울,2021-03-12,10.5,7.2,2353.0,14.9,1240.0
4449,108,서울,2021-03-13,9.2,4.6,650.0,14.2,1436.0
4450,108,서울,2021-03-14,9.1,4.9,701.0,14.7,1541.0


In [109]:
temp_seoul = df_seoul['평균기온(°C)'].values
temp_seoul = temp_seoul[::-1]
temp_seoul

array([ 10.6,   9.1,   9.2,  10.5,  10.2,   8.8,   6.6,   7. ,   6.2,
         6.6,   9. ,   7.2,   4.4,   2.3,   4.7,   7.8,   9.5,   8.2,
         4.2,   2.9,   0. ,   7.8,  10.8,   8.8,   1.6,  -5.8,  -8.3,
        -5.1,   1.2,   9.5,   7.6,   6.5,   5.5,   3.4,  -0.9,  -3.1,
         3.6,   6.8,   1.4,  -3.6,  -3.2,  -5.6,   5. ,   3.2,  -1.6,
        -8.8,  -2.6,   1.7,   5.3,   7.9,   7.3,   7.6,   5.5,   4.5,
        -0.1,  -6.3,  -3.8,  -5.5,  -4.9,   4.5,   1.7,   2. ,  -3.9,
        -6.7,  -7.7, -12.2, -14.9, -14.5,  -7.4,  -5.5,  -3.5,  -5.6,
        -5. ,  -4.2,  -8.9, -10.9,   0.1,   6.7,   5.8,   2.2,  -0.8,
         2.6,   4.5,   2. ,  -1. ,  -3.7,  -5.1,  -2.1,  -4.9,  -8.3,
        -8.3,  -7.7,  -1.1,   2.3,   5.3,   5.8,   1.5,  -0.5,   3.4,
         4.5,   2.1,  -0.6,   0.7,   2.4,   1.1,  -0.2,  -0.6,  -0.1,
         4.1,   5.6,   4.6,   2.9,   1.7,   5.6,   6.5,   2.8,  13.6,
        16.6,  15.9,  12.4,  11.1,  12.5,  12.2,  10.4,   9.3,   7.7,
         5.5,   8.5,

## 4. 코로나 DataFrame에 서울 기온 열 합치기

In [110]:
new_df = result.assign(seoulTemp = temp_seoul)
new_df

,accDefRate,accExamCnt,accExamCompCnt,careCnt,clearCnt,createDt,deathCnt,decideCnt,examCnt,resutlNegCnt,seq,stateDt,stateTime,updateDt,seoulTemp
0,1.3690604347,7084940,7013350,6588,87754,2021-03-15 09:51:09.37,1675,96017,71590,6917333,449,20210315,00:00,null,10.6
1,1.3668864032,7066401,6996558,6558,87408,2021-03-14 09:42:57.64,1669,95635,69843,6900923,448,20210314,00:00,null,9.1
2,1.3637663657,7046782,6978908,6884,86625,2021-03-13 09:36:41.886,1667,95176,67874,6883732,447,20210313,00:00,null,9.2
3,1.3635801994,7012664,6943926,7281,85743,2021-03-12 09:49:58.077,1662,94686,68738,6849240,446,20210312,00:00,null,10.5
4,1.3634619468,6976985,6908737,7871,84675,2021-03-11 09:39:02.159,1652,94198,68248,6814539,445,20210311,00:00,null,10.2
5,1.3646582727,6938884,6868606,7773,84312,2021-03-10 09:45:29.181,1648,93733,70278,6774873,444,20210310,00:00,null,8.8
6,1.3649942487,6902984,6832483,7718,83900,2021-03-09 09:52:46.909,1645,93263,70501,6739220,443,20210309,00:00,null,6.6
7,1.3660778742,6861809,6794415,7701,83474,2021-03-08 09:31:46.646,1642,92817,67394,6701598,442,20210308,00:00,null,7.0
8,1.3645371735,6843126,6776730,7617,83220,2021-03-07 09:52:34.246,1634,92471,66396,6684259,441,20210307,00:00,null,6.2
9,1.3624109264,6821943,6756772,7510,82913,2021-03-06 09:40:17.621,1632,92055,65171,6664717,440,20210306,00:00,null,6.6


In [131]:
# decideCnt가 누적 확진자 수 같아서 for문으로 일일 확진자를 구해보려는 시도 But 에러남
col = []
for i in range(370):
    dc = int(new_df.iloc[i]['decideCnt']) - int(new_df.iloc[i+1]['decideCnt'])
    col.append(dc)
col.insert(371,dc)
    
col

[382,
 459,
 490,
 488,
 465,
 470,
 446,
 346,
 416,
 418,
 398,
 424,
 444,
 344,
 355,
 355,
 415,
 387,
 395,
 440,
 356,
 332,
 416,
 448,
 561,
 621,
 621,
 457,
 343,
 326,
 362,
 403,
 504,
 444,
 303,
 288,
 371,
 393,
 370,
 451,
 467,
 335,
 305,
 355,
 453,
 469,
 497,
 559,
 349,
 437,
 392,
 431,
 345,
 399,
 403,
 385,
 119,
 790,
 580,
 512,
 524,
 554,
 536,
 451,
 664,
 641,
 674,
 868,
 838,
 714,
 1020,
 657,
 820,
 1028,
 967,
 1050,
 1044,
 807,
 970,
 1132,
 1240,
 985,
 1090,
 868,
 925,
 1098,
 1046,
 1064,
 1014,
 1075,
 880,
 718,
 1030,
 950,
 689,
 680,
 671,
 592,
 615,
 631,
 577,
 628,
 540,
 511,
 451,
 438,
 388,
 504,
 555,
 581,
 382,
 349,
 271,
 330,
 386,
 363,
 343,
 313,
 230,
 222,
 208,
 205,
 191,
 143,
 146,
 100,
 126,
 143,
 89,
 145,
 126,
 117,
 75,
 97,
 124,
 127,
 113,
 125,
 103,
 88,
 119,
 61,
 77,
 155,
 121,
 89,
 58,
 76,
 91,
 73,
 47,
 110,
 73,
 102,
 98,
 58,
 72,
 54,
 69,
 114,
 75,
 73,
 64,
 75,
 63,
 77,
 113,
 38,
 50,

In [132]:
# 일일 확진자 수 열에 추가하기
new_df = result.assign(dailyCnt = col)
new_df

,accDefRate,accExamCnt,accExamCompCnt,careCnt,clearCnt,createDt,deathCnt,decideCnt,examCnt,resutlNegCnt,seq,stateDt,stateTime,updateDt,dailyCnt
0,1.3690604347,7084940,7013350,6588,87754,2021-03-15 09:51:09.37,1675,96017,71590,6917333,449,20210315,00:00,null,382
1,1.3668864032,7066401,6996558,6558,87408,2021-03-14 09:42:57.64,1669,95635,69843,6900923,448,20210314,00:00,null,459
2,1.3637663657,7046782,6978908,6884,86625,2021-03-13 09:36:41.886,1667,95176,67874,6883732,447,20210313,00:00,null,490
3,1.3635801994,7012664,6943926,7281,85743,2021-03-12 09:49:58.077,1662,94686,68738,6849240,446,20210312,00:00,null,488
4,1.3634619468,6976985,6908737,7871,84675,2021-03-11 09:39:02.159,1652,94198,68248,6814539,445,20210311,00:00,null,465
5,1.3646582727,6938884,6868606,7773,84312,2021-03-10 09:45:29.181,1648,93733,70278,6774873,444,20210310,00:00,null,470
6,1.3649942487,6902984,6832483,7718,83900,2021-03-09 09:52:46.909,1645,93263,70501,6739220,443,20210309,00:00,null,446
7,1.3660778742,6861809,6794415,7701,83474,2021-03-08 09:31:46.646,1642,92817,67394,6701598,442,20210308,00:00,null,346
8,1.3645371735,6843126,6776730,7617,83220,2021-03-07 09:52:34.246,1634,92471,66396,6684259,441,20210307,00:00,null,416
9,1.3624109264,6821943,6756772,7510,82913,2021-03-06 09:40:17.621,1632,92055,65171,6664717,440,20210306,00:00,null,418


## 5. 예측 모델에 사용될 데이터 프레임 완성!